In [1]:
import pandas as pd
from sqlalchemy import create_engine, __version__
print("Pandas Version:", pd.__version__)
print("SQLAlachemy Version:", __version__)

Pandas Version: 2.2.2
SQLAlachemy Version: 2.0.30


In [2]:
def get_data(creds, query):
    """
    Connects to a PostgreSQL database using the given credentials and executes the given SQL query.

    Args:
        creds (dict): A dictionary containing the following keys:
            - PG_DB (str): The name of the database.
            - PG_USER (str): The username for the database.
            - PG_HOST (str): The hostname of the database.
            - PG_PSWD (str): The password for the database.
        query (str): The SQL query to be executed.

    Returns:
        pandas.DataFrame: A DataFrame containing the results of the query.

    Raises:
        None

    Notes:
        This function uses the `sqlalchemy` library to connect to the database and execute the query.
        The connection is closed after the query is executed.
    """
    
    # Connect to the database
    string_con = f"postgresql://{creds['PG_USER']}:{creds['PG_PSWD']}@{creds['PG_HOST']}/{creds['PG_DB']}"
    engine = create_engine(string_con)
    conn = engine.connect()

    # Get data
    df = pd.read_sql(query, conn)

    # Close connection
    conn.close()
    engine.dispose()

    return df

In [3]:
# Credentials
creds = {
    'PG_DB': 'pa004_5sdi',
    'PG_USER': 'meigabots',
    'PG_HOST': 'dpg-cpcf43rtg9os738cbgvg-a.oregon-postgres.render.com',
    'PG_PSWD': 'jHtR4eEQNDPquJjosnuZQ76KS6d1lMOT'
}

In [4]:
# Query to get data
query = '''
    SELECT
        d."ID" AS id, 
        d."GENDER" AS gender, 
        d."AGE" AS age, 
        v."DRIVING_LICENSE" AS driving_license, 
        d."REGION_CODE" AS region_code, 
        v."PREVIOUSLY_INSURED" AS previously_insured, 
        v."VEHICLE_AGE" AS vehicle_age, 
        v."VEHICLE_DAMAGE" AS vehicle_damage, 
        pr."ANNUAL_PREMIUM" AS annual_premium, 
        pr."POLICY_SALES_CHANNEL" AS policy_sales_channel, 
        pr."VINTAGE" AS vintage, 
        pr."RESPONSE" AS response
    FROM public.demographics d
    LEFT JOIN public.vehicle v ON (d."ID" = v."ID")
    LEFT JOIN public.policy_and_response pr ON (d."ID" = pr."ID")
'''

In [5]:
# Import data
df = get_data(creds, query)

In [6]:
# Save data
df.to_csv('../../data/raw/health_insurance_cross_sell.csv', index=False)